## Recommendation System
Collaborative filtering with implicit feedback based on latent factors. Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [23]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import pandas as pd
import numpy as np

from test_model import (get_patent_fields_list, get_ml_patents, 
                        create_title_abstract_col,trim_data, 
                        structure_dataframe, partition_dataframe, 
                        build_pipeline, process_docs, pat_inv_map, get_topics)

from rec_system import alphanum_to_int, int_to_alphanum

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import KMeans

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary, mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import requests
from bs4 import BeautifulSoup
import pickle
import math

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

In [25]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Data understanding - Acquire data

#### Data understanding - Acquire data for text workflows

In [26]:
# load pickled dataset
with open('/Users/lee/Documents/techniche/techniche/data/raw_data_1000', 'rb') as f:
    raw_data_1000 = pickle.load(f)

In [27]:
# define keys as criteria to subset dataset #1 for non-text workflows
retained_keys = ['patent_number', 'patent_firstnamed_assignee_id']

# subset raw dataset by desired keys/columns
data_1000 = trim_data(data=raw_data_1000, keys=retained_keys)

In [28]:
# define keys as criteria to subset dataset #2, for text workflows
retained_keys_2 = ['patent_number', 'patent_firstnamed_assignee_id',
                   'patent_title', 'patent_abstract']

# subset raw dataset by desired keys/columns for text analysis workflows
data_1000_2 = trim_data(data=raw_data_1000, keys=retained_keys_2)

#### Data preparation

In [29]:
# create new item in dataset #2 by concat of patent_title and patent_abstract
data_1000_2 = create_title_abstract_col(data=data_1000_2)

In [30]:
# create Pandas dataframe from dataset #1
df_1000 = pd.DataFrame(data_1000)

In [31]:
# create Pandas dataframe from dataset #2
df_1000_2 = pd.DataFrame(data_1000_2)

In [32]:
# for dataset #1: drop row that contains invalid data
df_1000[df_1000.patent_number.str.contains('[RE]')]
df_1000 = df_1000.drop(df_1000.index[[717]])

# drop NaNs in patent_firstnamed_assignee_id column
df_1000 = df_1000.dropna()

In [33]:
# for dataset#2: drop row that contains invalid data
df_1000_2[df_1000_2.patent_number.str.contains('[RE]')]
df_1000_2 = df_1000_2.drop(df_1000_2.index[[717]])

# drop NaNs in patent_firstnamed_assignee_id column
df_1000_2 = df_1000_2.dropna()

#### Data preparation - model #1
Prepare data on user-item relationships for each user-company in format that ALS can use.
We require each unique assignee ID in the rows of the matrix, and each unique item ID in columns of matrix.
Values of matrix should be (?) binary user-item preference * confidence

In [34]:
# create new rating column and assign value of 1
df_1000['rating'] = 1

In [35]:
# convert patent_number column from string to int
df_1000 = df_1000.astype({'patent_number': 'int64'})
# uncomment to confirm
# df_1000.info()

In [36]:
# convert alphanumeric patent_firstnamed_assignee_id col to int
df_1000 = df_1000.astype({'patent_number': 'int64'})

In [37]:
# df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs)
df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs) % 65536 # 2^16

In [38]:
# df_1000['patent_firstnamed_assignee_id'] = df_1000['patent_firstnamed_assignee_id'].apply(hash).apply(abs)
df_1000['patent_number'] = df_1000['patent_number'] % 65536 # 2^16

In [39]:
df_1000 = df_1000.astype({'patent_firstnamed_assignee_id': 'int'})

#### Data preparation - model #1 - create Spark dataframe from pandas dataframe

In [40]:
sp_df_1000 = spark.createDataFrame(df_1000)

In [41]:
# cast columns from bigint to int
sp_df_1000_2 = sp_df_1000.withColumn("patent_firstnamed_assignee_id", sp_df_1000["patent_firstnamed_assignee_id"].cast(IntegerType())).withColumn("patent_number", sp_df_1000["patent_number"].cast(IntegerType())).withColumn("rating", sp_df_1000["rating"].cast(IntegerType()))

In [42]:
# partition dataframe 
(training, test) = sp_df_1000.randomSplit([0.8, 0.2])

### Model # 1
Build the recommendation model using ALS on the training data

In [43]:
# build ALS recommendation model
als = ALS(maxIter=5,
          regParam=0.01, 
          rank=10, # number of latent topics- ME-10?
          alpha=30,
          implicitPrefs=True, # # implicitPrefs=True b/c ratings are implicit
          userCol="patent_firstnamed_assignee_id", 
          itemCol="patent_number", 
          ratingCol="rating",
          coldStartStrategy="nan") # coldStartStrategy="nan" to retain NaNs

In [44]:
# fit ALS model to the training set
model = als.fit(training)

#### Model #1 - Evaluation - Compare to naive baseline
Compare model evaluation result with naive baseline model that only outputs (for explicit - the average rating (or you may try one that outputs the average rating per movie).

#### Model #1 - Optimize model

In [ ]:
params = ParamGridBuilder().addGrid(als_model.regParam, [0.01,0.001,0.1]).addGrid(als_model.rank, [4,10,50]).build()


## instantiate crossvalidator estimator
cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(movie_ratings)    

#### Getting Predictions

In [49]:
# get predictions for test set
predictions = model.transform(test)

In [56]:
# get predictions for training set
predictions_train = model.transform(training)
predictions_train_df = predictions_train.toPandas()
predictions_train_df

,patent_firstnamed_assignee_id,patent_number,rating,prediction
0,2428,3997,1,0.979045
1,29959,9900,1,0.368814
2,2428,26087,1,0.979045
3,2428,32304,1,0.979045
4,26595,43256,1,0.368790
5,29202,34602,1,0.369658
6,14464,2525,1,0.880219
7,41979,3986,1,0.369143
8,36082,4042,1,0.369534
9,20767,27977,1,0.649512


In [52]:
# view in Pandas df
predictions_df = predictions.toPandas()
predictions_df

,patent_firstnamed_assignee_id,patent_number,rating,prediction
0,4197,54258,1,NaN
1,25717,44792,1,NaN
2,49660,4396,1,NaN
3,6180,15931,1,NaN
4,51402,58869,1,NaN
5,2428,14760,1,NaN
6,37381,22313,1,NaN
7,65328,4923,1,NaN
8,3298,60097,1,NaN
9,61893,40736,1,NaN


In [51]:
predictions_df.dropna()

,patent_firstnamed_assignee_id,patent_number,rating,prediction


In [ ]:
predictions = model.transform(training)

In [ ]:
predictions_train_df = predictions.toPandas()

In [ ]:
predictions_train_df

In [ ]:
- content-similarity
- limits of patent space
- TF-IDF vectorization of patents - metrics - avg distance between 
- distance between individual patents, with ranking
- take tf-idf vector and argsort by absolute value, to see which features are most
- important to this patent. 
- Get top 20 features. While normally would do cosine distance betweel all vectors. BUT,
- only do cosine distance between these top 20 features, for cold start patents

### Model #2 - Data preparation

In [ ]:
df_1000_2.info()

#### Model 2 - Data preparation - text data

In [ ]:
# instantiate TF-IDF Vectorizer using standard English stopwords
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
# fit TF-IDF matrix on text column
tfidf_matrix = tfidf.fit_transform(df_1000_2['patent_title_abstract'])

In [ ]:
# output matrix, 972 docs, 5364 terms
tfidf_matrix.shape

### Model 3 - compute distance metric

In [ ]:
# compute cosine similarity matrix between docs using linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# construct reverse map of indices and pat_title_abstract
indices = pd.Series(df_1000_2.index, index=df_1000_2['patent_title_abstract']).drop_duplicates()

In [ ]:
indices

In [ ]:
# tfidf vec requires list, not just string
unseen_data = 'computer vision natural language processor'
unseen_data=[unseen_data]

In [ ]:
unseen_tfidf = tfidf.transform(unseen_data)

#### Model #2 - Apply K means clustering to distance matrix

In [ ]:
km = KMeans(20)

In [ ]:
kmresult = km.fit(tfidf_matrix).predict(unseen_tfidf)

In [ ]:
kmresult_p = km.predict(unseen_tfidf)

In [ ]:
kmresult_p